In [1]:
import pandas as pd

df = pd.read_csv('RanCat_predictions_JFMAMJ(Final)_clv_with_avgvalues_seg.csv')

In [ ]:
import pandas as pd

def reason_for_churn(row): 
    if row['Predicted Status'] == 'Not Renewed':
        reasons = []

        # Policy Type Related
        if row['renewal type'] == 'TY Onwards':
            reasons.append("Policy having TY Onwards Renewal Type")
        if row['biztype'] == 'Roll Over':
            reasons.append("Roll Over Business Type")
        if row['tie up'] in ['HYUNDAI', 'Non-OEM', 'MIBL OEM']:
            reasons.append(f"Tie Up with {row['tie up']}")

        # Premium Related
        if row['total od premium'] > 6460.00:
            reasons.append("High Own-Damage Premium")
        if row['total premium payable'] > 21759.50:
            reasons.append("High Total Premium Payable")
        if row['before gst add-on gwp'] > 7971.50:
            reasons.append("High Add-On Premium")
        if row['total tp premium'] > 8597.00:
            reasons.append("High Third-Party Premium")
        if row['applicable discount with ncb'] > 77.50:
            reasons.append("High Discount with NCB")
        if row['applicable discount with ncb'] <= 62.50:
            reasons.append("Low Discount with NCB")

        # Renewal Rate Related
        if row['Renewal Rate Status'] == 'Increase':
            reasons.append("Higher Renewal Premium Impact")
        if row['Renewal Rate Status'] == 'No Change' and row['total premium payable'] > 11101.00:
            reasons.append("Renewal Rate No Change & High Total Premium Payable")

        # Claim Related
        if row['Claim Happaned/Not'] == 'Yes':
            reasons.append("Claims Happened")

        # Vehicle Related
        if row['vehicle idv'] <= 436905.00:
            reasons.append("Low Vehicle IDV")
        if row['vehicle idv'] > 987300.00 and row['total premium payable'] > 16731.00:
            reasons.append("High Vehicle IDV & High Total Premium")

        # Age Related (Updated with High-Preference Rules)
        if row['age'] <= 3.98:
            reasons.append("Young Vehicle Age")
        if row['age'] > 8.57:
            reasons.append("Old Vehicle Age")

        # Customer Related
        if row['ncb % previous year'] < 10:
            reasons.append("Low No Claim Bonus Percentage")
        if row['Customer Tenure'] < 1.5:
            reasons.append("Minimal Customer Tenure")

        # Policy-Wise Purchase
        if row['policy_wise_purchase'] < 2.5:
            reasons.append("Minimal Policies Purchased")

        return ', '.join(reasons) if reasons else 'Organic Churn'

    return ''

df['Not Renewed Reasons'] = df.apply(reason_for_churn, axis=1)

df.to_csv('RanCat_predictions_JFMAMJ(Final)_segment_reasons.csv', index=False)

print(df[['policy no', 'Not Renewed Reasons']].head())  


                   policy no  \
0  '201140020224700585500000   
1  '201150020622800005800000   
2  '201150030124700434600000   
3  '201150020121851293900000   
4  '201120010121800356400000   

                                 Not Renewed Reasons  
0  Roll Over Business Type, Tie Up with Non-OEM, ...  
1  Tie Up with Non-OEM, High Own-Damage Premium, ...  
2  Roll Over Business Type, Tie Up with Non-OEM, ...  
3  Tie Up with Non-OEM, High Own-Damage Premium, ...  
4  Tie Up with Non-OEM, High Own-Damage Premium, ...  


In [ ]:
import pandas as pd
import re
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, classification_report, log_loss
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

query = 'SELECT * FROM public.overall_cleaned_base_and_pr_ef_policyef;'
df = pd.read_sql(query, con=engine)

In [ ]:
# Ensure columns are of the correct data type
numeric_columns = [
    'before gst add-on gwp', 'total od premium', 'total tp premium', 'total premium payable', 
    'ncb % previous year', 'applicable discount with ncb', 'age', 'vehicle idv'
]

# Convert columns to numeric, coercing errors to NaN
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

def reason_for_churn(row): 
    if row['Policy Status'] == 'Not Renewed':
        reasons = []

        # Policy Type Related
        if row['renewal type'] == 'TY Onwards':
            reasons.append("Policy having TY Onwards Renewal Type")
        if row['biztype'] == 'Roll Over':
            reasons.append("Roll Over Business Type")
        if row['tie up'] in ['HYUNDAI', 'Non-OEM', 'MIBL OEM']:
            reasons.append(f"Tie Up with {row['tie up']}")

        # Premium Related
        if row['total od premium'] > 6460.00:
            reasons.append("High Own-Damage Premium")
        if row['total premium payable'] > 21759.50:
            reasons.append("High Total Premium Payable")
        if row['before gst add-on gwp'] > 7971.50:
            reasons.append("High Add-On Premium")
        if row['total tp premium'] > 8597.00:
            reasons.append("High Third-Party Premium")
        if row['applicable discount with ncb'] > 77.50:
            reasons.append("High Discount with NCB")
        if row['applicable discount with ncb'] <= 62.50:
            reasons.append("Low Discount with NCB")

        # Renewal Rate Related
        if row['Renewal Rate Status'] == 'Increase':
            reasons.append("Higher Renewal Premium Impact")
        if row['Renewal Rate Status'] == 'No Change' and row['total premium payable'] > 11101.00:
            reasons.append("Renewal Rate No Change & High Total Premium Payable")

        # Claim Related
        if row['Claim Happaned/Not'] == 'Yes':
            reasons.append("Claims Happened")

        # Vehicle Related
        if row['vehicle idv'] <= 436905.00:
            reasons.append("Low Vehicle IDV")
        if row['vehicle idv'] > 987300.00 and row['total premium payable'] > 16731.00:
            reasons.append("High Vehicle IDV & High Total Premium")

        # Age Related (Updated with High-Preference Rules)
        if row['age'] <= 3.98:
            reasons.append("Young Vehicle Age")
        if row['age'] > 8.57:
            reasons.append("Old Vehicle Age")

        # Customer Related
        if row['ncb % previous year'] < 10:
            reasons.append("Low No Claim Bonus Percentage")
        if row['Customer Tenure'] < 1.5:
            reasons.append("Minimal Customer Tenure")

        # Policy-Wise Purchase
        if row['policy_wise_purchase'] < 2.5:
            reasons.append("Minimal Policies Purchased")

        return ', '.join(reasons) if reasons else 'Organic Churn'

    return ''

# Apply the function to the DataFrame
df['Not Renewed Reasons'] = df.apply(reason_for_churn, axis=1)

# Export the results to a PostgreSQL table
output_table_name = "overall_cleaned_base_and_pr_ef_policyef_with_reasons"
df.to_sql(output_table_name, con=engine, if_exists='replace', index=False)

print(f"Data successfully saved to table '{output_table_name}' in the database.")

Data successfully saved to table 'overall_cleaned_base_and_pr_ef_policyef_with_reasons' in the database.
